### Test notebook to develop line activity module.
 - Open the NEID file
 - Identify the data extension and the wavelength extension
 - Choose the Na D lines and extract and plot them.
 - Figure out how to enter the line list into the config or some other file that


In [1]:
# Standard dependencies  (copied from the bias module)
import configparser
import numpy as np
from astropy.io import fits

# Pipeline dependencies
from kpfpipe.logger import start_logger
from kpfpipe.primitives.level0 import KPF0_Primitive
# from kpfpipe.models.level0 import KPF0

# External dependencies
from keckdrpframework.models.action import Action
from keckdrpframework.models.arguments import Arguments
from keckdrpframework.models.processing_context import ProcessingContext


In [2]:
#         #Start logger
#         self.logger=None
#         if not self.logger:
#             self.logger=self.context.logger
#         self.logger.info('Loading config from: {}'.format(self.config_path))


In [ ]:
# Open NEID file:
DIR="/Users/hisaacson/ownCloud/KPF-Pipeline-TestData/NEIDdata/TAUCETI/L1"
FILE="neidL1_20210719T232216.fits"